<a href="https://colab.research.google.com/github/wenchun0731/city-hunter/blob/main/FineTune(YOLOV8)(wen%E9%87%8D%E8%A3%BD%E7%89%883_0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#掛載雲端硬碟
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#下載ultralytics開源庫
!pip install ultralytics

In [4]:
#下載YOLOv8並檢查模型配置
from ultralytics import YOLO
import os
from IPython.display import display,Image
from IPython import display
display.clear_output()#清除当前输出单元格的所有输出内容
!yolo model=checks#检查 YOLO 模型的状态或配置信息。

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 761, in entrypoint
    check_dict_alignment(full_args_dict, {a: ""})
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 440, in check_dict_alignment
    raise SyntaxError(string + CLI_HELP_MSG) from e
SyntaxError: 'YOLO' is not a valid YOLO argument. 

    Arguments received: ['yolo', 'model=checks#检查', 'YOLO', '模型的状态或配置信息。']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of {'obb', 'classify', 'detect', 'segment', 'pose'}
                MODE (required) is one of {'export', 'predict', 'train', 'val', 'track', 'benchmark'}
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at https://docs.ul

In [7]:
#Deep sort
#DeepSort(使用自己的訓練檔)
!pip install ultralytics
!pip install deep-sort-realtime
import numpy as np
import cv2
import math
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab.patches import cv2_imshow

#輸入
model = YOLO('/content/drive/MyDrive/Finetune/Licence_Finetune/train/train/weights/best.pt')
cap = cv2.VideoCapture("/content/drive/MyDrive/Finetune/Licence_Finetune/test_video/IMG_0461.MOV")#輸入影片
fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fNUMS = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
videoWriter = cv2.VideoWriter("/content/drive/MyDrive/Finetune/Licence_Finetune/detect/predict.mp4", fourcc, fps, size)#輸出位置


#nms_max_overlap非極大值抑制（NMS）過程中允許的最大重疊比例
#nn_budget控制最近鄰搜尋中使用的樣本數量的上限
#min_hits: 設定物體被確認為有效跟蹤需要的最小幀數。增大 min_hits 可以避免追蹤器誤認為是有效的目標，從而減少漂移。
tracker = DeepSort(
    max_age=60,  # 增加此值以保持更長時間的追蹤
    max_cosine_distance=0.5, # 減少此值以提高特徵匹配的精確度一般設置為0到1之間
    nn_budget=100  # 調整最近鄰搜尋的樣本數量
)

# 定義顏色字典
colors = {
    'ambulance':(0,255,0),   # 綠色
    'bus': (0,130,255),     # 青色
    'car': (0,0,255),      # 藍色
    'fire truck': (255,0,0),  # 紅色
    'licence': (60,0,128),   # 紫色
    'person': (255,255,0),   # 黃色
    'police': (0,0,0),     # 黑色
    'traffic light': (255,0,255) # 粉紅色
}
rule_id=[]
violation_frame_count=[]
rule_plates=[]
class TrafficViolationDetector:
    def __init__(self, image_data, slope, intercept, frame_count):
        self.image_data = image_data

        self.slope = slope
        self.intercept = intercept
        self.frame_count=frame_count
    def detect_violations(self):
        self.data_sort()
        return self.data_sort()

    def data_sort(self):
      if 'person' in self.image_data and 'car' in self.image_data:
          for tag, data in self.image_data.items():
              if tag=='car' or tag=='person':

                for data_k, data_v in data.items():
                    # 確保 data_v 是列表而不是字符串
                    if isinstance(data_v, list):
                        for id, coordinate in enumerate(data_v):
                            # 檢查是否為有效的坐標
                            if len(coordinate) >= 2 and self.on_crosswalk(coordinate[0], coordinate[1]):
                                data_v[id] = coordinate
                            else:
                                data_v[id] = []
                        # 過濾空陣列
                        self.image_data[tag][data_k] = [item for item in data_v if item]
          return self.rule_judge()



    def on_crosswalk(self, x, y):
        if x>250 and y>400:
          low_y = (self.slope - 0.07) * x + (self.intercept - 25)
          up_y = (self.slope + 0.04) * x + (self.intercept + 70)
          return up_y > y > low_y

    def rule_judge(self):
        people_data = self.image_data['person']
        car_data = self.image_data['car']

        for c_k, c_v in car_data.items():

            is_violation = False
            for p_v in people_data.values():

                if is_violation:
                    break
                for c_data, p_data in [[c, p] for c in c_v for p in p_v]:

                    if self.distance_cal(c_data, p_data):
                        is_violation = True
                        violate_information=self.you_violation(c_k)
                        return violate_information
                        break

    def distance_cal(self, car, person):
        x_minus = abs(car[0] - person[0])
        y_minus = abs(car[1] - person[1])
        if y_minus == 0:
            distance = x_minus
        elif x_minus == 0:
            distance = y_minus
        else:
            distance = math.hypot(x_minus, y_minus)
        return distance >= 200

    def you_violation(self, id):
        if id not in rule_id:

            violation_frame_count.append(self.frame_count)
            #找到違規車輛的車牌是多少的函示
            plates_id=self.find_plates(id)
            if plates_id == None:
              return None
            else:
              rule_id.append(id)

              rule_plates.append(plates_id)
              return [self.frame_count,id,plates_id]
            # print(f'在第{self.frame_count}偵的時候，車輛{id}違規,他的車牌id是{plates_id}')
            # self.clear_list(self.image_data)

    # # 定義清空列表的函式
    # def clear_list(self,lst):
    #     lst.clear()  # 使用 clear() 方法清空列表

    def coor_min_max(self,what_coor):
        min_value = min(min(sublist) for sublist in what_coor)
        max_value = max(max(sublist) for sublist in what_coor)
        return min_value,max_value
    def find_plates(self,id):
        #從image_data['car'][id]找到車輛座標
        rule_car_coordinate=self.image_data['car'][id]
        if 'licence' in self.image_data:
          rule_plates_coordinate=self.image_data['licence']



          all_distance=[]
          car_min,car_max=self.coor_min_max(rule_car_coordinate)
          for plates_id,plates_coordinates in rule_plates_coordinate.items():
            plates_min,plates_max=self.coor_min_max(plates_coordinates)
            x_minus = abs(plates_min - car_min)
            y_minus = abs(plates_max - car_max)
            if y_minus == 0:
                distance = x_minus

            elif x_minus == 0:
                distance = y_minus
            else:
                distance = math.hypot(x_minus, y_minus)
            all_distance.append([plates_id,distance])
          cars_plates_id = min(all_distance, key=lambda x: x[1])
          return cars_plates_id[0]
        else:

          return None

#＃＃ 計數器
frame_count = 0

# 儲存追蹤框位置的字典
image_data = {}

def box_label(image, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    #int(box[0]): 左邊界的 x 坐標，即框的左側位置（水平坐標）。
    #int(box[1]): 上邊界的 y 坐標，即框的上側位置（垂直坐標）。
    #int(box[2]): 右邊界的 x 坐標，即框的右側位置（水平坐標）。
    #int(box[3]): 下邊界的 y 坐標，即框的下側位置（垂直坐標）。
    p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))

    # 畫出矩形框
    cv2.rectangle(image, p1, p2, color, thickness=3, lineType=cv2.LINE_AA)
    if label:
        w, h = cv2.getTextSize(label, 0, fontScale=2 / 3, thickness=3)[0]
        outside = p1[1] - h >= 3
        p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
        cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)
        cv2.putText(image,
                    label, (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
                    0, 2/3, txt_color, thickness=3, lineType=cv2.LINE_AA)

while cap.isOpened():
    success, frame = cap.read()

    if success:
        frame_count += 1  # 每幀加1
        results = model(frame, conf=0.5, iou=0.8)#置信值
        outputs = results[0].boxes.data.cpu().numpy()

        detections = []

        if outputs is not None:
            for output in outputs:
                x1, y1, x2, y2 = list(map(int, output[:4]))
                class_id = int(output[5])

                # 根據模型的類別對應，將class_id對應到具體的類別名稱
                if class_id == 0:
                    label = 'ambulance'
                elif class_id == 1:
                    label = 'bus'
                elif class_id == 2:
                    label = 'car'
                elif class_id == 3:
                    label = 'fire truck'
                elif class_id == 4:
                    label = 'licence'
                elif class_id == 5:
                    label = 'person'
                elif class_id == 6:
                    label = 'police'
                elif class_id == 7:
                    label = 'traffic light'

                color = colors.get(label, (128, 128, 128))  # 默認顏色

                detections.append(([x1, y1, int(x2-x1), int(y2-y1)], output[4], label))

            tracks = tracker.update_tracks(detections, frame=frame)

            for track in tracks:
                if not track.is_confirmed():
                    continue
                track_id = track.track_id #追蹤框id
                bbox = track.to_ltrb()

                # 獲取對應的顏色
                label = track.det_class
                color = colors.get(label, (128, 128, 128))  # 默認顏色
                box_label(frame, bbox, '#' + str(int(track_id)) + label, color)

                # 儲存追蹤框位置
                if frame_count % 60 == 0:

                  # 將角點格式化為所需的字串格式
                  p1 = [int(bbox[0]), int(bbox[1])]  # 左上角
                  p2 = [int(bbox[2]), int(bbox[3])]  # 右下角
                  image_position =  [
                        [p1[0], p1[1]],  # 左上角
                        [p1[0], p2[1]],  # 左下角
                        [p2[0], p2[1]],  # 右下角
                        [p2[0], p1[1]]   # 右上角
                    ]


                  # 檢查是否存在此label
                  if label not in image_data:
                    image_data[label] = {}  # 如果沒有此label，則新增一個字典

                  # 檢查此label下是否存在此track_id
                  if track_id not in image_data[label]:
                    image_data[label][track_id] = image_position


            image_data_copy=image_data.copy()
                  # rule_id = []
            if frame_count % 60 == 0:
                  slope=0.305
                  intercept=329.859
                  detector = TrafficViolationDetector(image_data, slope, intercept, frame_count)
                  violate_result=detector.detect_violations()
                  if violate_result is not None:
                    print('傳資料到資料庫')







            #  清空字典
            image_data.clear()
            image_data_copy.clear()


        cv2.putText(frame, "https://blog.csdn.net/zhaocj", (25, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        videoWriter.write(frame)

    else:
        break
print(rule_id)
print(rule_plates)
cap.release()
videoWriter.release()
cv2.destroyAllWindows()

串流輸出內容已截斷至最後 5000 行。

0: 384x640 2 ambulances, 40.9ms
Speed: 3.1ms preprocess, 40.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 ambulances, 1 car, 49.9ms
Speed: 3.0ms preprocess, 49.9ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ambulance, 1 car, 39.4ms
Speed: 7.2ms preprocess, 39.4ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ambulance, 1 car, 40.9ms
Speed: 3.0ms preprocess, 40.9ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ambulance, 1 car, 42.1ms
Speed: 6.1ms preprocess, 42.1ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ambulance, 1 car, 52.2ms
Speed: 3.0ms preprocess, 52.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 39.9ms
Speed: 3.0ms preprocess, 39.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ambulance, 1 car, 44.9ms
Speed: